# Building an agent with an episodic memory

The episodic memory is a type of long-term memory that allows us to remember specific events or experiences in our lives. Unlike semantic memory, which stores general knowledge and facts, episodic memory is more focused on personal experiences and the context in which they occurred.

In the context of HybridAGI we wanted to ground this concept into Computer Sciences, and what best representation than a memory that store the program traces?

In HybridAGI's Trace Memory, each action is vectorized and stored, so the system can retrieve past actions between sessions by using the `PastActionSearch` tool.

Note: To avoid recursive recall of memories, we don't store the `PastActionSearch` action.

## Setting up the Knowledge Base locally

HybridAGI works with a low-latency hybrid vector/graph database called [FalkorDB](https://www.falkordb.com/). This knowledge base should be started when you work with HybridAGI.

Start the database using docker with the following command in you terminal:

```bash
docker run -p 6379:6379 -p 3000:3000 -it --rm falkordb/falkordb:edge
```

Now let's build a simple example to illustrate the concept

In [9]:
import dspy
from hybridagi import HybridAGI

# Setup the LM
lm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n"])

dspy.configure(lm=lm)

agent = HybridAGI(
    agent_name="episodic_memory_agent",
)

agent.add_programs_from_folders(["programs/episodic_memory"])

# Now let's try our pipeline
prediction = agent.execute("What is a neuro-symbolic artificial intelligence?")

print(prediction.final_answer)

Neuro-Symbolic AI is a form of composite AI that combines machine learning methods and symbolic systems (for example, knowledge graphs) to create more robust and trustworthy AI models. This combination allows statistical patterns to be combined with explicitly defined rules and knowledge to give AI systems the ability to understand and reason about complex concepts in a human-like manner. It aims to achieve artificial general intelligence by augmenting and combining the strengths of statistical AI, like machine learning, with the capabilities of human-like symbolic knowledge and reasoning.


In [10]:
# Let's see how the system came up with the answer

print(prediction.program_trace)

 --- Step 0 ---
Call Program: main
Program Purpose: What is a neuro-symbolic artificial intelligence?
 --- Step 1 ---
Action Purpose: Search past actions to answer the objective's question
Action: {
  "query": "neuro symbolic AI definition",
  "past_actions": []
}
 --- Step 2 ---
Decision Purpose: Check if the answer to the objective's question is present in the previous actions search
Decision Question: Is the answer in the previous search? if the answer is not present or not relevant select websearch otherwise select answer
Decision: WEB_SEARCH
 --- Step 3 ---
Action Purpose: Perform a duckduckgo search
Action: {
  "search_query": "neuro symbolic AI definition",
  "results": [
    {
      "title": "Neuro-symbolic AI - Wikipedia",
      "href": "https://en.wikipedia.org/wiki/Neuro-symbolic_AI",
      "body": "Neuro-symbolic AI is a type of artificial intelligence that integrates neural and symbolic AI architectures to address the weaknesses of each, providing a robust AI capable of re

We can observe that the system tried to first remember previous actions

Because the trace memory is empty, it decided to perform an additional search using the `DuckDuckGoSearch` tool as instructed in the graph.

Now we can try to execute again the same question and see how it answer.

In [11]:
prediction = agent.execute("What is the definition of a Neuro-Symbolic AI?")

print(prediction.final_answer)

Neuro-Symbolic AI is a form of artificial intelligence that combines machine learning methods and symbolic systems (for example, knowledge graphs) to create more robust and trustworthy AI models. This combination allows statistical patterns to be combined with explicitly defined rules and knowledge to give AI systems the ability to understand and reason about complex concepts in a human-like manner. It aims to achieve artificial general intelligence by augmenting and combining the strengths of statistical AI, like machine learning, with the capabilities of human-like symbolic knowledge and reasoning.


In [12]:
# The answer seem correct, but let's be sure on how it came up with the answer

print(prediction.program_trace)

 --- Step 0 ---
Call Program: main
Program Purpose: What is the definition of a Neuro-Symbolic AI?
 --- Step 1 ---
Action Purpose: Search past actions to answer the objective's question
Action: {
  "query": "definition of Neuro-Symbolic AI",
  "past_actions": [
    {
      "past_action": "{\n  \"message\": \"Neuro-Symbolic AI is a form of composite AI that combines machine learning methods and symbolic systems (for example, knowledge graphs) to create more robust and trustworthy AI models. This combination allows statistical patterns to be combined with explicitly defined rules and knowledge to give AI systems the ability to understand and reason about complex concepts in a human-like manner. It aims to achieve artificial general intelligence by augmenting and combining the strengths of statistical AI, like machine learning, with the capabilities of human-like symbolic knowledge and reasoning.\"\n}"
    },
    {
      "past_action": "{\n  \"search_query\": \"neuro symbolic AI definitio

As you can see, the system now used the previous actions to answer this question, without needing to perform a websearch again

Feel free to explore more on this topic and experiment with HybridAGI graph program's and trace memory!